# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [ ]:
import pandas as pd

df_1 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv')
df_2 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv')
df_3 = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv')

for df in [df_1, df_2, df_3]:
    display(df)

In [ ]:
# renaming columns
for df in [df_1, df_2, df_3]:
    df.rename(columns=lambda x: x.replace(' ', '_').lower(), inplace=True)
    df.rename(columns={'st': 'state'}, inplace=True)

display(df_1, df_2, df_3)

In [ ]:
# concatenate the three dataframes
df = pd.concat([df_1, df_2, df_3])
display(df)

In [ ]:

# replace values
import re

def replace_values(value, replacements):
    if isinstance(value, str):
        for pattern in replacements:
            value = re.sub(pattern[0], pattern[1], value)
        return value
    return value

df['gender'] = df['gender'].apply(replace_values, args=[[['female|Femal', 'F'], ['Male', 'M']]])
df['state'] = df['state'].apply(replace_values, args=[[[r'Cali$', 'California'], ['WA', 'Washington'], ['AZ', 'Arizona']]])
df['education'] = df['education'].apply(replace_values, args=[[[r'Bachelors$', 'Bachelor']]])
df['customer_lifetime_value'] = df['customer_lifetime_value'].apply(replace_values, args=[[[r'%', '']]])
df['vehicle_class'] = df['vehicle_class'].apply(replace_values, args=[[[r'Sports.*|Luxury.*', 'Luxury']]])

# fill nas
df['number_of_open_complaints'] = df['number_of_open_complaints'].apply(replace_values, args=[[[r'\d+/(\d+)/\d+', r'\1']]])
df['number_of_open_complaints'] = df['number_of_open_complaints'].fillna(0)

# recast data types
data_types = {'customer': 'object',
          'state': 'category', 
          'gender': 'category', 
          'education': 'category', 
          'customer_lifetime_value': 'float', 
          'income': 'float',
          'monthly_premium_auto': 'float', 
          'number_of_open_complaints': 'int', 
          'policy_type': 'category', 
          'vehicle_class': 'category', 
          'total_claim_amount': 'float'}

df = df.astype(data_types)

# remove nulls

df= df.dropna(subset=['customer'])

# Fill categorical columns with the value 'Unknown'
categorical_cols = df.select_dtypes(include=['category']).columns
df[categorical_cols] = df[categorical_cols].astype('object')
df[categorical_cols] = df[categorical_cols].fillna('Unknown')
df[categorical_cols] = df[categorical_cols].astype('category') 

# Fill life time value, income and monthly premium auto with the mean
df['customer_lifetime_value'] = df['customer_lifetime_value'].fillna(df['customer_lifetime_value'].mean())
df['income'] = df['income'].fillna(df['income'].mean())
df['monthly_premium_auto'] = df['monthly_premium_auto'].fillna(df['monthly_premium_auto'].mean())

#Fill Total Claim Amount with 0
df['total_claim_amount'] = df['total_claim_amount'].fillna(0)


# remove duplicates

df = df.drop_duplicates()

df.drop_duplicates(subset=['customer'], keep='first', inplace=True)

df.head(500)


# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [ ]:
df_mkt = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv')
df_mkt.head()
df_mkt.columns 

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

In [ ]:

pivot_table = df_mkt.pivot_table(values='total_claim_amount', index='sales_channel', aggfunc='sum')

pivot_table = pivot_table.round(2)

display(pivot_table)

max_revenue_channel = pivot_table['total_claim_amount'].idxmax()
max_revenue = pivot_table['total_claim_amount'].max()

print(f"Best channel : {max_revenue_channel} with ${max_revenue} of revenue.")



2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [ ]:
clf_pivot_table = df_mkt.pivot_table(values='customer_lifetime_value', index=['gender','education'], aggfunc='mean')
display(clf_pivot_table)

max_clv = clf_pivot_table.max().max()
max_clv_id = clf_pivot_table.idxmax().max()
print(max_clv_id)
print(f"Max Customer Lifetime Value : {max_clv_id} with ${max_clv}")

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [ ]:
# Your code goes here
complaints_pivot_table = df_mkt.pivot_table(values='number_of_open_complaints', index=['month', 'policy_type'], aggfunc='sum').reset_index()
complaints_pivot_table = complaints_pivot_table.sort_values(by='number_of_open_complaints', ascending=False)
display(complaints_pivot_table)